<a href="https://colab.research.google.com/github/kanikachitnis1018/Summarizer/blob/main/flowchart_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch

In [2]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import re


In [3]:
text_to_be_split = """ Photosynthesis is the process by which plants convert sunlight into chemical energy. Chlorophyll in the leaves captures sunlight, allowing plants to produce glucose from carbon dioxide and water. This process is essential for life on Earth as it provides oxygen and energy for other organisms.

The water cycle describes the continuous movement of water on, above, and below the surface of the Earth. Water evaporates from oceans, lakes, and rivers, condenses in the atmosphere to form clouds, and falls back to the surface as precipitation. This cycle helps maintain ecosystems and supports life globally.

Newton’s laws of motion describe the relationship between a body and the forces acting on it. The first law states that an object will remain at rest or move at a constant velocity unless acted upon by a force. The second law quantifies force as mass times acceleration, and the third law states that every action has an equal and opposite reaction. """


paragraphs = text_to_be_split.split("\n\n")

paragraphs

[' Photosynthesis is the process by which plants convert sunlight into chemical energy. Chlorophyll in the leaves captures sunlight, allowing plants to produce glucose from carbon dioxide and water. This process is essential for life on Earth as it provides oxygen and energy for other organisms.',
 'The water cycle describes the continuous movement of water on, above, and below the surface of the Earth. Water evaporates from oceans, lakes, and rivers, condenses in the atmosphere to form clouds, and falls back to the surface as precipitation. This cycle helps maintain ecosystems and supports life globally.',
 'Newton’s laws of motion describe the relationship between a body and the forces acting on it. The first law states that an object will remain at rest or move at a constant velocity unless acted upon by a force. The second law quantifies force as mass times acceleration, and the third law states that every action has an equal and opposite reaction. ']

In [4]:
cleaned_paras = []

for para in paragraphs:
    para = para.strip()

    para = re.sub(r'\s+', ' ', para)

    para = re.sub(r'[^a-zA-Z0-9.,;:!?()\'" -]', '', para)

    cleaned_paras.append(para)

for i, para in enumerate(cleaned_paras, 1):
    print(f"Paragraph {i}: {para}\n")

Paragraph 1: Photosynthesis is the process by which plants convert sunlight into chemical energy. Chlorophyll in the leaves captures sunlight, allowing plants to produce glucose from carbon dioxide and water. This process is essential for life on Earth as it provides oxygen and energy for other organisms.

Paragraph 2: The water cycle describes the continuous movement of water on, above, and below the surface of the Earth. Water evaporates from oceans, lakes, and rivers, condenses in the atmosphere to form clouds, and falls back to the surface as precipitation. This cycle helps maintain ecosystems and supports life globally.

Paragraph 3: Newtons laws of motion describe the relationship between a body and the forces acting on it. The first law states that an object will remain at rest or move at a constant velocity unless acted upon by a force. The second law quantifies force as mass times acceleration, and the third law states that every action has an equal and opposite reaction.



In [16]:
import nltk
from nltk.tokenize import sent_tokenize
from itertools import chain
from transformers import BertTokenizer, BertModel

nltk.download("punkt_tab")

sentences = list(chain.from_iterable(map(sent_tokenize, cleaned_paras)))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

encoded = tokenizer(    # converts sentences to tokens
    sentences,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():   # converts tokens to token embeddings
    outputs = model(**encoded)
yeah
embeddings = outputs.last_hidden_state    # converts token embeddings to sentence embeddings
sentence_embeddings = embeddings.mean(dim=1)

sentences_per_para = list(map(sent_tokenize, cleaned_paras))
para_lengths = list(map(len, sentences_per_para))

sentence_groups = torch.split(sentence_embeddings, para_lengths)
